# Transformers 模型量化技术：AWQ

![img](https://huggingface.co/datasets/ybelkada/documentation-images/resolve/main/Thumbnail.png)

在2023年6月，Ji Lin等人发表了论文[AWQ：Activation-aware Weight Quantization for LLM Compression and Acceleration](https://arxiv.org/pdf/2306.00978.pdf)。

这篇论文详细介绍了一种激活感知权重量化算法，可以用于压缩任何基于 Transformer 的语言模型，同时只有微小的性能下降。关于 AWQ 算法的详细介绍，见[MIT Han Song 教授分享](https://hanlab.mit.edu/projects/awq)。

transformers 现在支持两个不同的 AWQ 开源实现库：

- [AutoAWQ](https://github.com/casper-hansen/AutoAWQ)
- [LLM-AWQ](https://github.com/mit-han-lab/llm-awq) 


因为 LLM-AWQ 不支持 Nvidia T4 GPU（课程演示 GPU），所以我们使用 AutoAWQ 库来介绍和演示 AWQ 模型量化技术。

In [1]:
import os
os.environ['HF_HOME'] = '/home/featurize/data/huggingface/'
os.environ['HF_HUB_CACHE'] = '/home/featurize/data/huggingface/hub/'
os.environ['HF_DATASETS_CACHE'] = '/home/featurize/data/huggingface/datasets/'

## 量化前模型测试文本生成任务

In [3]:
from transformers import pipeline

model_path = "facebook/opt-125m"

# 使用 GPU 加载原始的 OPT-125m 模型
generator = pipeline('text-generation',
                     model=model_path,
                     device=0,
                     do_sample=True,
                     num_return_sequences=3)

#### 实测GPU显存占用：加载 OPT-125m 模型后

```shell
Every 1.0s: nvidia-smi                                                                                          featurize: Sat Apr  6 12:31:53 2024

Sat Apr  6 12:31:53 2024
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 525.85.05    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:10:00.0 Off |                  N/A |
| 33%   38C    P8    25W / 350W |   1828MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|    0   N/A  N/A     39257      C   ...ent/miniconda3/bin/python     1826MiB |
+-----------------------------------------------------------------------------+

```

In [4]:
generator("The woman worked as a")

[{'generated_text': 'The woman worked as a waitress and he always brought his drink and "hot chocolate" with him.'},
 {'generated_text': 'The woman worked as a cashier at the nearby bakery and was also a cook for the owner of'},
 {'generated_text': 'The woman worked as a waitress at a restaurant for a long time. She always made everyone feel important'}]

In [5]:
generator("The man worked as a")

[{'generated_text': 'The man worked as a machinist for the Pentagon for a long time as well, so this'},
 {'generated_text': 'The man worked as a carwash operator and auto repair shop operator while the woman works in the company'},
 {'generated_text': 'The man worked as a waiter for a bank after working in it for 10 years. Before being a'}]

## 使用 AutoAWQ 量化模型

下面我们以 `facebook opt-125m` 模型为例，使用 `AutoAWQ` 库实现的 AWQ 算法实现模型量化。

In [6]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


quant_path = "models/opt-125m-awq"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 11.02it/s]


In [7]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
Generating validation split: 214670 examples [00:07, 28859.64 examples/s]
AWQ: 100%|██████████| 12/12 [01:44<00:00,  8.68s/it]


#### 实测GPU显存使用：量化模型时峰值达到将近 4GB

```shell
Every 1.0s: nvidia-smi                                                                                          featurize: Sat Apr  6 12:33:05 2024

Sat Apr  6 12:33:05 2024
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.05    Driver Version: 525.85.05    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:10:00.0 Off |                  N/A |
| 33%   51C    P2   156W / 350W |   4244MiB / 24576MiB |      72%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU   GI   CI        PID   Type   Process name                  GPU Memory |
|        ID   ID                                                   Usage      |
|=============================================================================|
|    0   N/A  N/A     39257      C   ...ent/miniconda3/bin/python     1960MiB |
+-----------------------------------------------------------------------------+
```

In [8]:
quant_config

{'zero_point': True, 'q_group_size': 128, 'w_bit': 4, 'version': 'GEMM'}

#### Transformers 兼容性配置

为了使`quant_config` 与 transformers 兼容，我们需要修改配置文件：`使用 Transformers.AwqConfig 来实例化量化模型配置`

In [9]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config

In [10]:
# 保存模型权重
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)  # 保存分词器

('models/opt-125m-awq/tokenizer_config.json',
 'models/opt-125m-awq/special_tokens_map.json',
 'models/opt-125m-awq/vocab.json',
 'models/opt-125m-awq/merges.txt',
 'models/opt-125m-awq/added_tokens.json',
 'models/opt-125m-awq/tokenizer.json')

### 使用 GPU 加载量化模型

In [11]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(quant_path)
model = AutoModelForCausalLM.from_pretrained(quant_path, device_map="cuda").to(0)

In [12]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)


In [13]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)

Merry Christmas! I'm glad to see you today
That's, a thanks
that, that... a - and my, a, I'm
thank you, and what, a, is to, a, a b, a?, be, a, a, a, a, and a, a, a


In [17]:
result = generate_text("how to import the body health ")
print(result)

how to import the body health   how to do this, how to import my body health? body health?

1


how to import the body health?

i can to import the body body weight info

you can get the body weight by add the body weight


this is i can add the body weight


## Homework：使用 AWQ 算法量化 Facebook OPT-2.7B 模型

Facebook OPT 模型：https://huggingface.co/facebook?search_models=opt